This Notebook is for plotting parts of the PAP algorithm, to compare with the original paper by Pouchou and Pichoir (1991).

In [ ]:
import hyperspy.api as hs
import numpy as np
import plotly.graph_objects as go

from PAP_functions.PAP_helper_functions import (theoretical_energy, 
    get_C_A_Z_arrays,
    wt2at,
    at2wt)
from PAP_functions.PAP_area_F import (set_m_small,
    ionization_cross_section_Q,
    mean_atomic_mass_M,
    mean_ionzation_potential_J,
    energy_dependent_terms_f_of_v,
    energy_loss_dE_drhos,
    whole_dE_drhos,
    deceleration_factor_one_over_S,
    backscattering_factor_R,
    mean_atomic_number_Zb,
    mean_backscattering_coefficient_eta,
    area_F)
from PAP_functions.PAP_parameterization import (parameterization_phi_of_rhoz,absorption_correction)

In [ ]:
hs.material.mass_absorption_coefficient(element='Ga', energies='O_Ka')

In [ ]:
## Looking at the values

elements = ['Ga', 'Sb']
line = 'Ga_La'
e0 = 15  # keV
concentrations = [0.5, 0.5]  # at%

e_c = theoretical_energy(line=line)
u = e_c / e0
m_small = set_m_small(line=line)

array_C, array_A, array_Z = get_C_A_Z_arrays(elements=elements, 
                                         concentrations=concentrations, concentration_type='at')
q = ionization_cross_section_Q(e0=e0, line=line)
m = mean_atomic_mass_M(array_C=array_C, array_Z=array_Z, array_A=array_A)
j = mean_ionzation_potential_J(array_C=array_C, array_Z=array_Z, array_A=array_A)
f_of_v = energy_dependent_terms_f_of_v(e=e0, j=j)
dE_drhos = energy_loss_dE_drhos(m=m, j=j, f_of_v=f_of_v)
dE_drhos2 = whole_dE_drhos(array_C=array_C, array_Z=array_Z, array_A=array_A, e0=e0)

print(f'C wt%: {array_C}')
print(f'A Da: {array_A}')
print(f'Z: {array_Z}')
print(f'Q: {q:.3f}')
print(f'M: {m:.3f}')
print(f'J: {j:.3f}')
print(f'1/f(V): {1/f_of_v:.3f}')

In [ ]:
def plot_dE_drhos(*, fig, elements, at_list, energies, name):
    array_C, array_A, array_Z = get_C_A_Z_arrays(elements=elements, 
                                         concentrations=at_list, concentration_type='at')
    m = mean_atomic_mass_M(array_C=array_C, array_Z=array_Z, array_A=array_A)
    j = mean_ionzation_potential_J(array_C=array_C, array_Z=array_Z, array_A=array_A)
    f_of_v_list = [energy_dependent_terms_f_of_v(e=e, j=j) for e in energies]
    de_drhos_list = np.array([energy_loss_dE_drhos(m=m, j=j, f_of_v=f) for f in f_of_v_list])
    fig.add_trace(go.Scatter(x=energies, y=de_drhos_list, name=name))


In [ ]:
energies = np.linspace(0.01, 50, 10000)
# f_of_v_list = [energy_dependent_terms_f_of_v(e=e, j=j) for e in energies]
# de_drhos_list = np.array([energy_loss_dE_drhos(m=m, j=j, f_of_v=f) for f in f_of_v_list])

fig = go.Figure()
plot_dE_drhos(fig=fig, elements=['Ga', 'Sb'], at_list=[0.5, 0.5], energies=energies, name='GaSb')
plot_dE_drhos(fig=fig, elements=['Ga', 'As'], at_list=[0.5, 0.5], energies=energies, name='GaAs')
plot_dE_drhos(fig=fig, elements=['Cu'], at_list=[1.], energies=energies, name='Cu (PAP Fig. 4)')

fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"),
                  xaxis_title="Energy [keV]", xaxis_type="log",
                  yaxis_title="dE/d\u03C1s [keV cm<sup>2</sup>/g]",
                  margin=dict(l=5, r=5, b=5, t=5), width=800, height=400,
                  legend=dict(x=0.995, y=0.99, xanchor='right',),
                  ) 

In [ ]:
# fig.write_image('../mastersthesis/figures/PAP_energy_loss_dE_drhos.pdf')
# # fig.write_image('../mastersthesis/figures/PAP_deceleration_of_electrons.pdf')  # old name


In [ ]:
# Making a Q(U) plot

e0_max = 30
def add_line_q_of_u(fig, line, name):
    e_c = theoretical_energy(line=line)
    e0_list = np.arange(e_c, e0_max, 0.01)
    u_list = e0_list / e_c
    q_list = [ionization_cross_section_Q(e0=e0, line=line) for e0 in e0_list]
    print('max q =', round(max(q_list),3), ' at U0 =' , round(u_list[q_list.index(max(q_list))],3), 
    ' at E =', round(e0_list[q_list.index(max(q_list))], 3), 'keV')
    fig.add_trace(go.Scatter(x=u_list, y=q_list, name=f'{name}, E<sub>C</sub>{e_c:.2f} keV'))

fig = go.Figure()
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"))
add_line_q_of_u(fig, 'Ga_La', 'Ga L\u03B1')
add_line_q_of_u(fig, 'As_La', 'As L\u03B1')
fig.update_layout(xaxis_title='Overvoltage, U = E<sub>0</sub>/E<sub>c</sub>', yaxis_title='Ionization cross-section, Q(U)', # <sub> l</sub><sup>A</sup>
                  margin=dict(l=5, r=5, b=5, t=5), width=800, height=400,
                  legend=dict(x=0.995, y=0.99, xanchor='right',),
                  xaxis_range=[-0.1, 25])

fig.add_annotation(x=3.38, y=0.372, text="Q<sub> max</sub> at  (3.38, 0.37)<br> i.e. E<sub>0</sub> = 3.7 keV", ax=275, ay=0)
fig.add_annotation(x=3.38, y=0.273, text="Q<sub> max</sub> at  (3.38, 0.27)<br> i.e. E<sub>0</sub> = 4.3 keV", ax=275, ay=130)

# fig.write_image('../mastersthesis/figures/PAP_ionization_cross_section.pdf')

fig 

In [ ]:
# calculate R for all Z, with varying U0

array_U = [1.1, 1.3, 1.5, 2, 3, 5, 20]
array_all_Z = np.arange(1,90)
array_all_R = []
for z in array_all_Z:
    array_C = [1]
    array_Z = [z]
    array_R = []
    for u in array_U:
        array_R.append(backscattering_factor_R(u0=u, array_C=array_C, array_Z=array_Z))
    array_all_R.append(array_R)    
array_all_R = np.array(array_all_R)

fig = go.Figure()
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"),
                xaxis_title="Z", yaxis_title="Backscattering factor R", legend_title='Overvoltage',
                margin=dict(l=5, r=5, b=5, t=5), width=800, height=400,)

for i, u0 in enumerate(array_U):
    fig.add_trace(go.Scatter(x=array_all_Z, y=array_all_R[:,i], name=f"{u0}"))


# fig.write_image('../mastersthesis/figures/PAP_backscattering_factor.pdf')

fig

In [ ]:
# Surface ionization

# calculation of phi_zero
def phi_zero__surface_ionization_2(*, e0: float, e_c: float, array_C: np.array, array_Z: np.array):
    zb = mean_atomic_number_Zb(array_C=array_C, array_Z=array_Z)
    eta = mean_backscattering_coefficient_eta(zb=zb)
    u = e0 / e_c
    return 1 + 3.3 * (1 - 1 / (u**(2 - 2.3 * eta))) * eta**1.2

def plot_phi_zero():
    # remaking plot 24 from the paper
    e0_list = [1.1, 1.3, 1.5, 2, 3, 5, 20, 100]
    e_c = 1 # one to get the overvoltages in the paper
    all_z = np.arange(1, 90, 1)
    array_C = [1]
    fig = go.Figure()
    for e0 in e0_list:
        y = [phi_zero__surface_ionization_2(e0=e0, e_c=e_c, array_C=array_C, array_Z=[z]) for z in all_z]
        fig.add_trace(go.Scatter(x=all_z, y=y, name=f'{e0}'))
    return fig

fig = plot_phi_zero()
fig.update_layout(xaxis_title='Z', yaxis_title='Surface ionization potential, \u03C6(0)',
                    legend=dict(traceorder='reversed', title='Overvoltage'))
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"),
                    margin=dict(l=5, r=5, b=5, t=5), width=800, height=400)

# fig.write_image('../mastersthesis/figures/PAP_surface_ionization_potential.pdf')
fig

# Plotting $\phi(\rho z)$

Image from Goldstein, Fig 19.9, with



In [ ]:
rhoz = np.linspace(0, 120e-6, 10000)

def get_phi_of_rhoz(e0, line, elements):
    concentrations = np.ones_like(elements, dtype=float)
    return parameterization_phi_of_rhoz(e0=e0, line=line, elements=elements, 
                                        wt_concentrations=concentrations, use_q='divide', rhoz=rhoz)


def add_trace(e0, line, elements, name):
    rho = hs.material.elements[elements[0]].Physical_properties['density (g/cm^3)']
    # z = np.linspace(0, 10e-6, 10000)
    # rho_z = rho * z
    concentrations = np.ones_like(elements, dtype=float)
    phi = parameterization_phi_of_rhoz(e0=e0, line=line, elements=elements, 
                                        wt_concentrations=concentrations, use_q='divide', rhoz=rhoz)

    fig.add_trace(go.Scatter(x=rhoz*rho, y=phi, name=name))


fig = go.Figure()
add_trace(e0=15, line='Al_Ka', elements=['Al'], name='Al_Ka in Al')
add_trace(e0=15, line='Al_Ka', elements=['Ti'], name='Al_Ka in Ti')
add_trace(e0=15, line='Cu_Ka', elements=['Cu'], name='Cu_Ka in Cu')

fig.update_layout(width=700, height=400, #xaxis_range=[0,2e-4],
                  xaxis_title='rho * z',
                  margin=dict(l=5, r=5, b=50, t=5),)
fig

In [ ]:
hs.material.mass_absorption_coefficient(element='Cu', energies='Al_Ka')

In [ ]:
rhoz = np.linspace(0, 3e-5, 10000)

fig = go.Figure()
add_trace(e0=30, line='Cu_Ka', elements=['Cu'], name='Cu_Ka in Cu, 30kV')
add_trace(e0=15, line='Cu_Ka', elements=['Cu'], name='Cu_Ka in Cu, 15kV')
add_trace(e0=10, line='Cu_Ka', elements=['Cu'], name='Cu_Ka in Cu, 10kV')

fig.update_layout(width=700, height=400,
                  xaxis_title='rho * z',
                  margin=dict(l=5, r=5, b=50, t=5),)
fig

In [ ]:
rhoz = np.linspace(0, 2e-3, 10000)

fig = go.Figure()
add_trace(e0=30, line='Ga_La', elements=['Ga'], name='30kV')
add_trace(e0=15, line='Ga_La', elements=['Ga'], name='15kV')
add_trace(e0=10, line='Ga_La', elements=['Ga'], name='10kV')

fig.update_layout(width=700, height=400,
                  xaxis_title='\u03C1z', yaxis_title='\u03C6(\u03C1z)',
                  legend=dict(x=.995, y=0.99, xanchor='right', title='Ga L\u03B1 in Ga'),
                  margin=dict(l=5, r=5, b=50, t=5),)

fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"),
                    margin=dict(l=5, r=5, b=5, t=5), width=800, height=400)

# fig.write_image('../mastersthesis/figures/PAP_phi_of_rhoz.pdf')
fig

In [ ]:
rhoz = np.linspace(0, 200e-6, 10000)
fig = go.Figure()
phi = parameterization_phi_of_rhoz(e0=20, line='Al_Ka', elements=['Cu'], 
                                    wt_concentrations=[1.], use_q='divide', rhoz=rhoz)

mu_rho = hs.material.mass_absorption_mixture(
    elements=['Cu'],
    weight_percent=np.array([1]) * 100,
    energies=theoretical_energy('Al_Ka'),
)
print(mu_rho)

chi = mu_rho / np.sin(np.deg2rad(35.))  # csc of TOA
# chi = 100 / np.sin(np.deg2rad(35.))  # csc of TOA

print(chi)
phi_chi = phi * np.exp(-chi * rhoz)

fig.add_trace(go.Scatter(x=rhoz, y=phi, name='Al_Ka in Cu, generated'))
fig.add_trace(go.Scatter(x=rhoz, y=phi_chi, name='Al_Ka in Cu, emitted'))


fig.update_layout(width=700, height=400, #xaxis_range=[0,2e-4],
                  xaxis_title='rho * z',
                  margin=dict(l=5, r=5, b=50, t=5),)

print(f'Sum of phi: {np.sum(phi*rhoz)}')
print(f'Sum of phi(chi): {np.sum(phi_chi*rhoz)}')

ar_C, ar_A, ar_Z = get_C_A_Z_arrays(elements=['Cu'], concentrations=[1.])
print(f'F Al_Ka in Cu: {area_F(array_C=ar_C, array_A=ar_A, array_Z=ar_Z, line="Al_Ka", e0=20)}')
print(absorption_correction(e0=20, line='Al_Ka', elements=['Cu'], wt_concentrations=[1.], use_q='divide'))
fig

In [ ]:
fig = go.Figure()
energies = np.linspace(0.1, 10, 10000)

mu_rho = hs.material.mass_absorption_mixture(
    elements=['Cu'],
    weight_percent=np.array([1]) * 100,
    energies=energies,
)
fig.add_scatter(x=energies, y=mu_rho, name='Cu')
# add vline at Al_Ka
fig.add_vline(x=theoretical_energy('Al_Ka'), line_width=1, line_dash="dash", line_color="black")
fig.update_layout(yaxis_type='log')

'f'

## Trying to reproduce the figures of phi(rhoz) from the paper

They have plotted:

- Mg Ka in Al at 25 kV (Fig. 1)
- C K in C at 4-12 kV (Fig. 2)
- C K in W at 4-12 kV (Fig. 3)
- Cu Ka in Al at 29 kV (Fig. 7)
- Al Ka in Al at 10 kV (Fig. 17, which is really a 300 nm Al layer on W, Ag, Cu, Al and B, where the continous curve is the Al layer on Al)

Picture of the five figures is included in the cell below (might be deactivated when my NTNU account is deactivated).


![Image of Yaktocat](https://folk.ntnu.no/brynjamm/PAP_curves---phi_of_rhoz.png)

In [ ]:
# $ \phi (\rho z) = A \cdot \exp(- a \cdot (\rho z)) + (B \cdot (\rho z) + \phi(0) - A) \cdot \exp(- b \cdot (\rho z)) $
def phi_rhoz(*, rhoz: np.ndarray, A: float, a: float, B: float, b: float, phi_zero: float) -> np.ndarray:
    return A * np.exp(- a * rhoz) + (B * rhoz + phi_zero - A) * np.exp(- b * rhoz)

def phi_rhoz_2(*, elements: list, concentrations: np.ndarray, line: str, e0: float, rhoz: np.ndarray):
    # gives the absorption correction factor for a given line and composition
    list_C, list_A, list_Z = get_C_A_Z_lists(elements=elements, concentrations=concentrations)
    u = e0 / theoretical_energy(line)
    mu_rho = hs.material.mass_absorption_mixture(elements=elements, weight_percent=concentrations*100, 
                                                energies=theoretical_energy(line))
    zb = calculate_zb(list_C=list_C, list_Z=list_Z)
    eta = calculate_eta(zb=zb)
    big_f = calculate_F(elements=elements, line=line, concentrations=list_C, e0=e0)
    phi_zero = phi_zero__surface_ionization(u=u, eta=eta)
    r_bar = r_bar__average_depth_of_ionization(f=big_f, u=u, z_b=zb)
    slope_p = slope_P(f=big_f, r_bar=r_bar, zb=zb, u=u)
    b = factor_small_b(r_bar=r_bar, big_f=big_f, phi_zero=phi_zero)
    a = factor_small_a(p=slope_p, b=b, big_f=f, phi_zero=phi_zero, r_bar=r_bar)
    epsilon = factor_epsilon(a=a, b=b)
    big_b = factor_big_b(b=b, big_f=big_f, epsilon=epsilon, p=slope_p, phi_zero=phi_zero)
    big_a = factor_big_a(b=b, big_f=big_f, epsilon=epsilon, phi_zero=phi_zero, big_b=big_b)
    chi = mu_rho / np.sin(np.deg2rad(35))  # cosecant of 35 degrees
    f_of_chi = calc_f_of_chi(chi=chi, phi_zero=phi_zero, big_b=big_b, b=b, big_a=big_a, epsilon=epsilon)

    # phi of (rho z)
    print(f'A: {big_a:>9.2e}, \ta: {a:>9.2e}, \tB: {big_b:>9.2e}, \tb: {b:>9.2e}, \t\u03C6(0): {phi_zero:>9.2e}, \t\u03B5: {epsilon:>9.2e}')
    phi = phi_rhoz(rhoz=rhoz, A=big_a, a=a, B=big_b, b=b, phi_zero=phi_zero)
    return phi

def plot_phi_of_rhoz(*, element: str, line: str, name: str, e0: float, max_rhoz: float = 1e-3, fig: go.Figure = None):
    if fig is None:
        fig = go.Figure()
    rhoz = np.linspace(0, max_rhoz, 10000+1)
    phi = phi_rhoz_2(elements=[element], concentrations=np.ndarray([1.0]), line=line, e0=e0, rhoz=rhoz)
    fig.add_trace(go.Scatter(x=rhoz, y=phi, mode='lines', name=name, showlegend=True))
    fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"),
                    margin=dict(l=5, r=5, b=5, t=5), width=800, height=400,
                    # legend=dict(x=0.995, y=0.99, xanchor='right'), #,title='Ga L\u03B1 from GaAs'),
                    yaxis_title='\u03C6 (\u03C1z)', xaxis_title='\u03C1z [g/cm<sup>2</sup>]')
    return fig


In [ ]:
# - Mg Ka in Al at 25 kV (Fig. 1)

fig = go.Figure()
print('Is supposed to go to 0 at 1.5 mg/cm^2, i.e.', 1.5e-3 )
print('As of now (04.05.23), the plot is not correct.')
plot_phi_of_rhoz(element='Mg', line='Mg_Ka', name='Mg K\u03B1 in Al at 25 kV', e0=25, max_rhoz=3.5e-3, fig=fig)
plot_phi_of_rhoz(element='Mg', line='Mg_Ka', name='Mg K\u03B1 in Al at 15 kV', e0=15, max_rhoz=3.5e-3, fig=fig)

In [ ]:
# - C K in C at 4-12 kV (Fig. 2)

fig = go.Figure()
print('4 kV goes to zero around 60, and 8 kV around 180 um/cm^2, i.e.', 60e-6, 180e-6)
print('phi(0) looks Ok, but it does not go to a max, and the shape is completely wrong.')
e0_list = [4, 6, 8, 10, 12]
for e0 in e0_list:
    plot_phi_of_rhoz(element='C', line='C_Ka', name=f'C K\u03B1 in C at {e0} kV', e0=e0, max_rhoz=200e-3, fig=fig)
fig.update_layout(yaxis_range=[-0.5, 2.5])
fig

In [ ]:
# - C K in W at 4-12 kV (Fig. 3)


fig = go.Figure()
print('4 kV goes to zero around 110, and 8 kV around 300 um/cm^2, i.e.', 110e-6, 300e-6)
print('The values go completely off the chart, waaaaay to low.')
print('Again, phi(0) looks Ok, but it does not go to a max, and the shape is completely wrong.')
e0_list = [4, 6, 8, 10, 12]
for e0 in e0_list:
    plot_phi_of_rhoz(element='W', line='C_Ka', name=f'C K\u03B1 in C at {e0} kV', e0=e0, max_rhoz=0.05, fig=fig)

fig.update_layout(yaxis_range=[-10, 5])
fig


In [ ]:
# - Cu Ka in Al at 29 kV (Fig. 7)


fig = go.Figure()
print('Is supposed to go to 0 at 1.5 mg/cm^2, i.e.', 1.5e-3 )
print('As of now (04.05.23), the plot is not correct.')
print('The max is too high, the shape is too sharp, and the zero is off by 1e3 (goes to zero at 15e-6 and not 15e-3)')
plot_phi_of_rhoz(element='Al', line='Cu_Ka', name='Cu K\u03B1 in Al at 29 kV', e0=29, max_rhoz=17e-6, fig=fig)

In [ ]:
# - Al Ka in Al at 10 kV (Fig. 17, which is really a 300 nm Al layer on W, Ag, Cu, Al and B, where the continous curve is the Al layer on Al)

fig = go.Figure()
print('Is supposed to go to 0 at 0.3 mg/cm^2, i.e.', 0.3e-3 )
print('As of now (04.05.23), the plot is not correct.')
print('The max is too high, and the end point is 0.6 instead of 0.3 mg/cm^2')
plot_phi_of_rhoz(element='Al', line='Al_Ka', name='Al K\u03B1 in Al at 10 kV', e0=10, max_rhoz=8e-4, fig=fig)